<a href="https://colab.research.google.com/github/Karan1106/Assignment3/blob/master/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [3]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os
import random
from scipy import ndarray

Data Augmentation using opencv (Jeans)



In [ ]:
from scipy import ndimage, misc
import numpy as np
import os
import cv2

def rotate():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Jeans"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/jeans"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_rotate = cv2.imread(input_path)

        # rotate the image
        img_rotate_90_clockwise = cv2.rotate(image_to_rotate, cv2.ROTATE_90_CLOCKWISE)


        # create full output path, 'example.jpg' 
        # becomes 'rotate_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'rotated_'+image_path)
        cv2.imwrite(fullpath, img_rotate_90_clockwise)

if __name__ == '__main__':
    rotate()

In [ ]:
def resize():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Jeans"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/jeans"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_resize = cv2.imread(input_path)

        # resize the image
        image_resized = cv2.resize(image_to_resize, (300, 200))

        image_rgb = cv2.cvtColor(image_to_resize, cv2.COLOR_BGR2RGB)

        # create full output path, 'example.jpg' 
        # becomes 'resize_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'resize_'+image_path)
        cv2.imwrite(fullpath, image_resized)

if __name__ == '__main__':
    resize()

In [ ]:

def flip():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Jeans"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/jeans"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_flip = cv2.imread(input_path)

        # flip the image
        img_flip_ud = cv2.flip(image_to_flip, 0)
        


        # create full output path, 'example.jpg' 
        # becomes 'flip_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'flip_'+image_path)
        cv2.imwrite(fullpath, img_flip_ud)

if __name__ == '__main__':
    flip()

In [ ]:
def sharpen():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Jeans"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/jeans"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_sharp = cv2.imread(input_path)

        # sharp the image
        kernel = np.array([[0, -1, 0], 
                   [-1, 5,-1], 
                   [0, -1, 0]])
        

        # Sharpen image
        image_sharp = cv2.filter2D(image_to_sharp, -1, kernel)
        
        # create full output path, 'example.jpg' 
        # becomes 'sharp_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'sharp_'+image_path)
        cv2.imwrite(fullpath, image_sharp)

if __name__ == '__main__':
    sharpen()

In [ ]:
def bright():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Jeans"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/jeans"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_bright = cv2.imread(input_path)

        # bright the image
        image= cv2.add(image_to_bright,np.array([50.0]))
        


        # create full output path, 'example.jpg' 
        # becomes 'bright_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'bright_'+image_path)
        cv2.imwrite(fullpath, image)

if __name__ == '__main__':
    bright()

In [ ]:
#decreasing brightness
def decbright():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Jeans"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/jeans"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_bright = cv2.imread(input_path)

        # bright the image
        image= cv2.add(image_to_bright,np.array([-50.0]))
        


        # create full output path, 'example.jpg' 
        # becomes 'bright_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'decbright_'+image_path)
        cv2.imwrite(fullpath, image)

if __name__ == '__main__':
    decbright()

In [ ]:
def rotate():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Jeans"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/jeans"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_rotate = cv2.imread(input_path)

        # rotate the image
        height, width= image_to_rotate.shape[:2]
        rotation_matrix=cv2.getRotationMatrix2D((width/2, height/2), -30, 0.5)
        rotated=cv2.warpAffine(image_to_rotate, rotation_matrix, (width, height))

        fullpath = os.path.join(outPath, 'rotated_'+image_path)
        cv2.imwrite(fullpath, rotated)

if __name__ == '__main__':
    rotate()

In [ ]:
def rotate():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Jeans"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/jeans"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_rotate = cv2.imread(input_path)

        # rotate the image
        height, width= image_to_rotate.shape[:2]
        rotation_matrix=cv2.getRotationMatrix2D((width/2, height/2), 30, 0.5)
        rotated=cv2.warpAffine(image_to_rotate, rotation_matrix, (width, height))

        fullpath = os.path.join(outPath, 'rotated_'+image_path)
        cv2.imwrite(fullpath, rotated)

if __name__ == '__main__':
    rotate()

Data Augmentation using opencv (sarees)


In [ ]:
def rotate():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Saree"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/saree"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_rotate = cv2.imread(input_path)

        # rotate the image
        img_rotate_90_clockwise = cv2.rotate(image_to_rotate, cv2.ROTATE_90_CLOCKWISE)


        # create full output path, 'example.jpg' 
        # becomes 'rotate_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'rotated_'+image_path)
        cv2.imwrite(fullpath, img_rotate_90_clockwise)

if __name__ == '__main__':
    rotate()

In [ ]:
def bright():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Saree"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/saree"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_bright = cv2.imread(input_path)

        # bright the image
        image= cv2.add(image_to_bright,np.array([50.0]))
        


        # create full output path, 'example.jpg' 
        # becomes 'bright_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'bright_'+image_path)
        cv2.imwrite(fullpath, image)

if __name__ == '__main__':
    bright()

In [ ]:

def decbright():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Saree"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/saree"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_bright = cv2.imread(input_path)

        # bright the image
        image= cv2.add(image_to_bright,np.array([-50.0]))
        


        # create full output path, 'example.jpg' 
        # becomes 'bright_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'decbright_'+image_path)
        cv2.imwrite(fullpath, image)

if __name__ == '__main__':
    decbright()

In [ ]:
def flip():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Saree"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/saree"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_flip = cv2.imread(input_path)

        # flip the image
        img_flip_ud = cv2.flip(image_to_flip, 0)
        


        # create full output path, 'example.jpg' 
        # becomes 'flip_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'flip_'+image_path)
        cv2.imwrite(fullpath, img_flip_ud)

if __name__ == '__main__':
    flip()

In [ ]:

def resize():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Saree"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/saree"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_resize = cv2.imread(input_path)

        # resize the image
        image_resized = cv2.resize(image_to_resize, (300, 200))

        image_rgb = cv2.cvtColor(image_to_resize, cv2.COLOR_BGR2RGB)

        # create full output path, 'example.jpg' 
        # becomes 'resize_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'resize_'+image_path)
        cv2.imwrite(fullpath, image_resized)

if __name__ == '__main__':
    resize()

In [ ]:
def sharpen():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Saree"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/saree"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_sharp = cv2.imread(input_path)

        # sharp the image
        kernel = np.array([[0, -1, 0], 
                   [-1, 5,-1], 
                   [0, -1, 0]])
        

        # Sharpen image
        image_sharp = cv2.filter2D(image_to_sharp, -1, kernel)
        
        # create full output path, 'example.jpg' 
        # becomes 'sharp_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'sharp_'+image_path)
        cv2.imwrite(fullpath, image_sharp)

if __name__ == '__main__':
    sharpen()

In [ ]:
def rotate():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Saree"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/saree"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_rotate = cv2.imread(input_path)

        # rotate the image
        height, width= image_to_rotate.shape[:2]
        rotation_matrix=cv2.getRotationMatrix2D((width/2, height/2), 30, 0.5)
        rotated=cv2.warpAffine(image_to_rotate, rotation_matrix, (width, height))

        fullpath = os.path.join(outPath, 'rotated_'+image_path)
        cv2.imwrite(fullpath, rotated)

if __name__ == '__main__':
    rotate()

In [ ]:
def rotate():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/Saree"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/saree"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_rotate = cv2.imread(input_path)

        # rotate the image
        height, width= image_to_rotate.shape[:2]
        rotation_matrix=cv2.getRotationMatrix2D((width/2, height/2), -30, 0.5)
        rotated=cv2.warpAffine(image_to_rotate, rotation_matrix, (width, height))

        fullpath = os.path.join(outPath, 'rotated_'+image_path)
        cv2.imwrite(fullpath, rotated)

if __name__ == '__main__':
    rotate()

Data Augmentation using opencv (trousers)

In [ ]:
def rotate():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/trousers"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/trousers"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_rotate = cv2.imread(input_path)

        # rotate the image
        img_rotate_90_clockwise = cv2.rotate(image_to_rotate, cv2.ROTATE_90_CLOCKWISE)


        # create full output path, 'example.jpg' 
        # becomes 'rotate_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'rotated_'+image_path)
        cv2.imwrite(fullpath, img_rotate_90_clockwise)

if __name__ == '__main__':
    rotate()

In [ ]:
def resize():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/trousers"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/trousers"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_resize = cv2.imread(input_path)

        # resize the image
        image_resized = cv2.resize(image_to_resize, (300, 200))

        image_rgb = cv2.cvtColor(image_to_resize, cv2.COLOR_BGR2RGB)

        # create full output path, 'example.jpg' 
        # becomes 'resize_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'resize_'+image_path)
        cv2.imwrite(fullpath, image_resized)

if __name__ == '__main__':
    resize()

In [ ]:
def flip():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/trousers"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/trousers"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_flip = cv2.imread(input_path)

        # flip the image
        img_flip_ud = cv2.flip(image_to_flip, 0)
        


        # create full output path, 'example.jpg' 
        # becomes 'flip_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'flip_'+image_path)
        cv2.imwrite(fullpath, img_flip_ud)

if __name__ == '__main__':
    flip()

In [ ]:
def sharpen():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/trousers"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/trousers"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_sharp = cv2.imread(input_path)

        # sharp the image
        kernel = np.array([[0, -1, 0], 
                   [-1, 5,-1], 
                   [0, -1, 0]])
        

        # Sharpen image
        image_sharp = cv2.filter2D(image_to_sharp, -1, kernel)
        
        # create full output path, 'example.jpg' 
        # becomes 'sharp_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'sharp_'+image_path)
        cv2.imwrite(fullpath, image_sharp)

if __name__ == '__main__':
    sharpen()

In [ ]:
def bright():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/trousers"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/trousers"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_bright = cv2.imread(input_path)

        # bright the image
        image= cv2.add(image_to_bright,np.array([50.0]))
        


        # create full output path, 'example.jpg' 
        # becomes 'bright_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'bright_'+image_path)
        cv2.imwrite(fullpath, image)

if __name__ == '__main__':
    bright()

In [ ]:
def decbright():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/trousers"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/trousers"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_bright = cv2.imread(input_path)

        # bright the image
        image= cv2.add(image_to_bright,np.array([-50.0]))
        


        # create full output path, 'example.jpg' 
        # becomes 'bright_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'decbright_'+image_path)
        cv2.imwrite(fullpath, image)

if __name__ == '__main__':
    decbright()

In [ ]:
def rotate():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/trousers"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/trousers"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_rotate = cv2.imread(input_path)

        # rotate the image
        height, width= image_to_rotate.shape[:2]
        rotation_matrix=cv2.getRotationMatrix2D((width/2, height/2), -30, 0.5)
        rotated=cv2.warpAffine(image_to_rotate, rotation_matrix, (width, height))

        fullpath = os.path.join(outPath, 'rotated_'+image_path)
        cv2.imwrite(fullpath, rotated)

if __name__ == '__main__':
    rotate()

In [ ]:
def rotate():
    outPath = "/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images/trousers"
    path = "/content/gdrive/My Drive/Colab Notebooks/Scrap_train/trousers"

    # iterate through the names of contents of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the file
        input_path = os.path.join(path, image_path)
        image_to_rotate = cv2.imread(input_path)

        # rotate the image
        height, width= image_to_rotate.shape[:2]
        rotation_matrix=cv2.getRotationMatrix2D((width/2, height/2), 30, 0.5)
        rotated=cv2.warpAffine(image_to_rotate, rotation_matrix, (width, height))

        fullpath = os.path.join(outPath, 'rotated_'+image_path)
        cv2.imwrite(fullpath, rotated)

if __name__ == '__main__':
    rotate()

BUILDING CONVOLUTIONAL NEURAL NETWORK

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras import backend as K

Using TensorFlow backend.


In [4]:
# dimensions of our images.
#input_shape=(256,256,3)
img_width, img_height = 150, 150


trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images")
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="/content/gdrive/My Drive/Colab Notebooks/Scrap_test")

Found 1842 images belonging to 3 classes.
Found 308 images belonging to 3 classes.


In [5]:

nb_train_samples=1842
nb_validation_samples=308
batch_size=32
epochs=20

In [6]:
#model building
model = Sequential()
model.add(Conv2D(input_shape=(256,256,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))

#flatten layer
model.add(Flatten()) # Output convert into one dimension layer and will go to Dense layer
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Dense(3, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [7]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 128)      

SGD OPTIMIZER


In [9]:
from keras.callbacks import Callback
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import SGD

checkpoint= ModelCheckpoint("Modelsgd.h5", monitor="val_accuracy",mode='max', save_best_only=True,verbose=1)

earlystop= EarlyStopping(monitor="val_accuracy",min_delta=0,patience=5,verbose=1,restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor="val_accuracy", factor=0.2, patience=5, verbose=1, min_delta=0.001)

#putiing callbacks into callback list
callbacks = [earlystop, checkpoint, reduce_lr]
momentums = [0.0, 0.5, 0.9, 0.99]
opt = SGD(lr=0.01, momentum=0.5, decay=0.01)
#we use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [10]:
history=model.fit_generator(
    traindata,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=testdata,
    validation_steps=nb_validation_samples//batch_size,
    callbacks = [earlystop, checkpoint, reduce_lr])


Epoch 1/20
57/57 [==============================] - 533s 9s/step - loss: 1.1588 - accuracy: 0.4039 - val_loss: 1.0858 - val_accuracy: 0.3403

Epoch 00001: val_accuracy improved from -inf to 0.34028, saving model to Modelsgd.h5
Epoch 2/20
57/57 [==============================] - 56s 989ms/step - loss: 1.0416 - accuracy: 0.5453 - val_loss: 1.2472 - val_accuracy: 0.3587

Epoch 00002: val_accuracy improved from 0.34028 to 0.35870, saving model to Modelsgd.h5
Epoch 3/20
57/57 [==============================] - 51s 897ms/step - loss: 0.9817 - accuracy: 0.5287 - val_loss: 0.3769 - val_accuracy: 0.6739

Epoch 00003: val_accuracy improved from 0.35870 to 0.67391, saving model to Modelsgd.h5
Epoch 4/20
57/57 [==============================] - 51s 898ms/step - loss: 0.6485 - accuracy: 0.6878 - val_loss: 0.3274 - val_accuracy: 0.9058

Epoch 00004: val_accuracy improved from 0.67391 to 0.90580, saving model to Modelsgd.h5
Epoch 5/20
57/57 [==============================] - 51s 898ms/step - loss: 0

ADAM

In [11]:
from keras.callbacks import Callback
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint= ModelCheckpoint("Modeladam.h5", monitor="val_accuracy",mode='max', save_best_only=True,verbose=1)

earlystop= EarlyStopping(monitor="val_accuracy",min_delta=0,patience=5,verbose=1,restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor="val_accuracy", factor=0.2, patience=5, verbose=1, min_delta=0.001)

#putiing callbacks into callback list
callbacks = [earlystop, checkpoint, reduce_lr]
opt = Adam(lr=0.01, decay=0.01)
#we use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [12]:
history=model.fit_generator(
    traindata,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=testdata,
    validation_steps=nb_validation_samples//batch_size,
    callbacks = [earlystop, checkpoint, reduce_lr])

Epoch 1/20
57/57 [==============================] - 56s 977ms/step - loss: 50219267918.2038 - accuracy: 0.3481 - val_loss: 1.1013 - val_accuracy: 0.3507

Epoch 00001: val_accuracy improved from -inf to 0.35069, saving model to Modeladam.h5
Epoch 2/20
57/57 [==============================] - 54s 952ms/step - loss: 1.1050 - accuracy: 0.3387 - val_loss: 1.0875 - val_accuracy: 0.3623

Epoch 00002: val_accuracy improved from 0.35069 to 0.36232, saving model to Modeladam.h5
Epoch 3/20
57/57 [==============================] - 54s 954ms/step - loss: 1.1036 - accuracy: 0.3265 - val_loss: 1.0897 - val_accuracy: 0.3478

Epoch 00003: val_accuracy did not improve from 0.36232
Epoch 4/20
57/57 [==============================] - 54s 952ms/step - loss: 1.1051 - accuracy: 0.3282 - val_loss: 1.1549 - val_accuracy: 0.3152

Epoch 00004: val_accuracy did not improve from 0.36232
Epoch 5/20
57/57 [==============================] - 54s 951ms/step - loss: 1.1037 - accuracy: 0.3188 - val_loss: 1.0953 - val_acc

NADAM

In [13]:
from keras.callbacks import Callback
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint= ModelCheckpoint("Modelnadam.h5", monitor="val_accuracy",mode='max', save_best_only=True,verbose=1)

earlystop= EarlyStopping(monitor="val_accuracy",min_delta=0,patience=5,verbose=1,restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor="val_accuracy", factor=0.2, patience=5, verbose=1, min_delta=0.001)

#putiing callbacks into callback list
callbacks = [earlystop, checkpoint, reduce_lr]
opt = Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
#we use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [14]:
history=model.fit_generator(
    traindata,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=testdata,
    validation_steps=nb_validation_samples//batch_size,
    callbacks = [earlystop, checkpoint, reduce_lr])

Epoch 1/20
57/57 [==============================] - 60s 1s/step - loss: 1.1060 - accuracy: 0.3331 - val_loss: 1.1221 - val_accuracy: 0.3125

Epoch 00001: val_accuracy improved from -inf to 0.31250, saving model to Modelnadam.h5
Epoch 2/20
57/57 [==============================] - 58s 1s/step - loss: 1.1045 - accuracy: 0.3243 - val_loss: 1.0975 - val_accuracy: 0.3587

Epoch 00002: val_accuracy improved from 0.31250 to 0.35870, saving model to Modelnadam.h5
Epoch 3/20
57/57 [==============================] - 58s 1s/step - loss: 1.1051 - accuracy: 0.3144 - val_loss: 1.0976 - val_accuracy: 0.3587

Epoch 00003: val_accuracy did not improve from 0.35870
Epoch 4/20
57/57 [==============================] - 58s 1s/step - loss: 1.1014 - accuracy: 0.3381 - val_loss: 1.1030 - val_accuracy: 0.3261

Epoch 00004: val_accuracy did not improve from 0.35870
Epoch 5/20
57/57 [==============================] - 58s 1s/step - loss: 1.1002 - accuracy: 0.3354 - val_loss: 1.1245 - val_accuracy: 0.2717

Epoch 00

RMSprop

In [15]:
from keras.callbacks import Callback
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import SGD

checkpoint= ModelCheckpoint("Modelrmsprop.h5", monitor="val_accuracy",mode='max', save_best_only=True,verbose=1)

earlystop= EarlyStopping(monitor="val_accuracy",min_delta=0,patience=5,verbose=1,restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor="val_accuracy", factor=0.2, patience=5, verbose=1, min_delta=0.001)

#putiing callbacks into callback list
callbacks = [earlystop, checkpoint, reduce_lr]
momentums = [0.0, 0.5, 0.9, 0.99]
opt = RMSprop(lr=0.01)
#we use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [16]:
history=model.fit_generator(
    traindata,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=testdata,
    validation_steps=nb_validation_samples//batch_size,
    callbacks = [earlystop, checkpoint, reduce_lr])

Epoch 1/20
57/57 [==============================] - 59s 1s/step - loss: 1.1688 - accuracy: 0.3448 - val_loss: 1.1059 - val_accuracy: 0.3403

Epoch 00001: val_accuracy improved from -inf to 0.34028, saving model to Modelrmsprop.h5
Epoch 2/20
57/57 [==============================] - 59s 1s/step - loss: 1.0990 - accuracy: 0.3492 - val_loss: 1.1033 - val_accuracy: 0.3152

Epoch 00002: val_accuracy did not improve from 0.34028
Epoch 3/20
57/57 [==============================] - 58s 1s/step - loss: 1.1003 - accuracy: 0.3377 - val_loss: 1.0830 - val_accuracy: 0.3442

Epoch 00003: val_accuracy improved from 0.34028 to 0.34420, saving model to Modelrmsprop.h5
Epoch 4/20
57/57 [==============================] - 58s 1s/step - loss: 1.0997 - accuracy: 0.3396 - val_loss: 1.1104 - val_accuracy: 0.3333

Epoch 00004: val_accuracy did not improve from 0.34420
Epoch 5/20
57/57 [==============================] - 59s 1s/step - loss: 1.0989 - accuracy: 0.3442 - val_loss: 1.1064 - val_accuracy: 0.3659

Epoc

 Nesterov Accelerated Gradient Descent



In [8]:
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import SGD

checkpoint= ModelCheckpoint("ModelNag.h5", monitor="val_accuracy",mode='max', save_best_only=True,verbose=1)

earlystop= EarlyStopping(monitor="val_accuracy",min_delta=0,patience=5,verbose=1,restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor="val_accuracy", factor=0.2, patience=5, verbose=1, min_delta=0.001)

#putiing callbacks into callback list
callbacks = [earlystop, checkpoint, reduce_lr]
opt = SGD(lr=0.01, nesterov=True)
#we use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [9]:
history=model.fit_generator(
    traindata,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=testdata,
    validation_steps=nb_validation_samples//batch_size,
    callbacks = [earlystop, checkpoint, reduce_lr])


Epoch 1/20
57/57 [==============================] - 1284s 23s/step - loss: 1.5978 - accuracy: 0.3939 - val_loss: 1.0820 - val_accuracy: 0.4062

Epoch 00001: val_accuracy improved from -inf to 0.40625, saving model to ModelNag.h5
Epoch 2/20
57/57 [==============================] - 83s 1s/step - loss: 1.0609 - accuracy: 0.4376 - val_loss: 1.0569 - val_accuracy: 0.6739

Epoch 00002: val_accuracy improved from 0.40625 to 0.67391, saving model to ModelNag.h5
Epoch 3/20
57/57 [==============================] - 64s 1s/step - loss: 1.0144 - accuracy: 0.4989 - val_loss: 1.0434 - val_accuracy: 0.6739

Epoch 00003: val_accuracy did not improve from 0.67391
Epoch 4/20
57/57 [==============================] - 64s 1s/step - loss: 0.9019 - accuracy: 0.5558 - val_loss: 0.9011 - val_accuracy: 0.6993

Epoch 00004: val_accuracy improved from 0.67391 to 0.69928, saving model to ModelNag.h5
Epoch 5/20
57/57 [==============================] - 65s 1s/step - loss: 0.8305 - accuracy: 0.5872 - val_loss: 0.7552